# Wine Quality — Support Vector Regression (SVR) (one command per code cell)
# RBF-kernel SVR; scale features and tune C/gamma/epsilon for best results.


# Imports

In [ ]:
import pandas as pd  # data handling


In [ ]:
import numpy as np  # numeric helpers


In [ ]:
from sklearn.model_selection import train_test_split, cross_validate  # split and CV


In [ ]:
from sklearn.preprocessing import StandardScaler  # scaling


In [ ]:
from sklearn.svm import SVR  # support vector regressor


In [ ]:
from sklearn.pipeline import Pipeline  # scaler + model


In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score  # metrics


# Load dataset

In [ ]:
DATASET = 'red'  # 'red' or 'white'


In [ ]:
CSV_PATH = f"/Users/sb/Documents/Study-Material/Machine-Learning/winequality-{DATASET}.csv"  # dataset path


In [ ]:
df = pd.read_csv(CSV_PATH, sep=';')  # load CSV


# Split

In [ ]:
X = df.drop(columns=['quality'])  # features


In [ ]:
y = df['quality']  # target


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)  # split


# Pipeline: StandardScaler + SVR(RBF)

In [ ]:
svr = Pipeline([('scale', StandardScaler()), ('svr', SVR(kernel='rbf', C=1.0, gamma='scale', epsilon=0.1))])  # build pipeline


In [ ]:
svr.fit(X_train, y_train)  # fit pipeline


In [ ]:
pred = svr.predict(X_test)  # predict


# Evaluate

In [ ]:
mae = mean_absolute_error(y_test, pred)  # MAE


In [ ]:
rmse = mean_squared_error(y_test, pred, squared=False)  # RMSE


In [ ]:
r2 = r2_score(y_test, pred)  # R^2


In [ ]:
{'MAE': mae, 'RMSE': rmse, 'R2': r2}  # metrics


# Cross-validation

In [ ]:
cv = cross_validate(svr, X, y, cv=5, scoring=('r2','neg_root_mean_squared_error'))  # 5-fold CV


In [ ]:
{k: (np.mean(v), np.std(v)) for k, v in cv.items() if k.startswith('test_')}  # CV summary


# MLflow Tracking — log params, metrics, and model

In [ ]:
import mlflow  # experiment tracking


In [ ]:
import mlflow.sklearn  # sklearn flavor logging


In [ ]:
mlflow.set_experiment('SVR-WineQuality')  # experiment per algorithm


In [ ]:
mlflow.start_run(run_name=f"{DATASET}")  # start run


In [ ]:
mlflow.log_params({'C': svr.named_steps['svr'].C, 'gamma': svr.named_steps['svr'].gamma, 'epsilon': svr.named_steps['svr'].epsilon, 'kernel': svr.named_steps['svr'].kernel})  # params


In [ ]:
mlflow.log_metric('MAE', mae)  # MAE


In [ ]:
mlflow.log_metric('RMSE', rmse)  # RMSE


In [ ]:
mlflow.log_metric('R2', r2)  # R2


In [ ]:
mlflow.sklearn.log_model(svr, artifact_path='model')  # log model


In [ ]:
mlflow.end_run()  # end run


# Persistence — save to disk and reload

In [ ]:
import os  # fs utils


In [ ]:
import joblib  # persistence


In [ ]:
from datetime import datetime  # timestamp


In [ ]:
os.makedirs('Machine-Learning/models/SVR', exist_ok=True)  # ensure dir


In [ ]:
MODEL_PATH = f"Machine-Learning/models/SVR/svr_wine_{DATASET}_" + datetime.now().strftime('%Y%m%d_%H%M%S') + ".joblib"  # path


In [ ]:
joblib.dump(svr, MODEL_PATH)  # save


In [ ]:
loaded = joblib.load(MODEL_PATH)  # load


In [ ]:
float(loaded.predict(X_test.iloc[[0]])[0])  # quick check


# Streamlit App — minimal predictor UI

In [ ]:
APP_CODE = """
import joblib, pandas as pd, streamlit as st
st.set_page_config(page_title='Wine — SVR', page_icon='🍷')
st.title('🍷 Wine Quality — Support Vector Regression')
MODEL_PATH = st.text_input('Model path', value=r'%s')
if MODEL_PATH and st.button('Load model'):
    try:
        st.session_state['model'] = joblib.load(MODEL_PATH)
        st.success('Model loaded')
    except Exception as e:
        st.error(f'Load failed: {e}')
if 'model' in st.session_state:
    cols = %s
    vals = {c: st.number_input(c, value=0.0) for c in cols}
    if st.button('Predict quality'):
        df = pd.DataFrame([vals])
        try:
            y = st.session_state['model'].predict(df)
            st.metric('Predicted quality', f"{float(y[0]):.3f}")
        except Exception as e:
            st.error(f'Predict failed: {e}')
""" % (MODEL_PATH, list(X.columns))


In [ ]:
open('app_streamlit_wine.py', 'w').write(APP_CODE)


# AWS S3 Upload (commented)

In [ ]:
# import boto3
# s3 = boto3.client('s3')
# bucket = os.environ.get('S3_BUCKET','your-bucket')
# key = 'wine-models/SVR/' + os.path.basename(MODEL_PATH)
# s3.upload_file(MODEL_PATH, bucket, key)


In [ ]:
float(svr.predict(X_test.iloc[[0]])[0])  # single-row prediction
